In [1]:
import pandas as pd
import pickle

"""
Class to store the relevant information 
associated with a flower
"""
class flower:
	# values is the row of the sheet corresponding to this flower
	# self.hex is hex id
	# self.gene is the unique gene string of this flower
	# self.color is this flower's color
	def __init__(self, values, species):
		self.hex = values[0]
		self.gene = values[1]
		self.species = species
		#self.color = values[7]
		if "Rose" in self.species:
			self.color = values[8]
		else:
			self.color = values[7]
		self.values = values # To be deleted later?

	# Make printing pretty
	def __str__(self):
		return str(self.hex) + "\t" + str(self.gene) + "\t" + str(self.color) + "\t" + str(self.species)

	# True if flower2 is a potential parent/child of self
	def isParent(self, flower2):
		f2gene = flower2.gene
		return sum([self.isParentGene(self.gene[2 * x] + self.gene[2 * x + 1], f2gene[2 * x] + f2gene[2 * x + 1]) for x in range(len(flower2.gene) // 2)]) == (len(flower2.gene) // 2)

	# True if gene1 is a potential parent/child of gene2
	def isParentGene(self, gene1, gene2):
		gene1, gene2 = fix_case(gene1), fix_case(gene2)
		return gene1[0] == gene2[0] or gene1[1] == gene2[1]

	def seed(self):
		return "(seed)" in self.color

def fix_pickle():
	keys = ['Hyacinth', 'Lilies', 'Mums', 'Pansies', 'Roses', 'Tulips', 'Windflowers']
	new_flowers_dict = {}
	for key in keys:
		df = pd.read_csv(key + '.csv')
		flowers = []
		for x in df.values:
			flowers.append(flower(x, key))
		new_flowers_dict[key] = {f.hex: f for f in flowers}
	with open('flowers.pkl', 'wb') as f2:
		pickle.dump(new_flowers_dict, f2)
	return new_flowers_dict

# This is all of the data from the google sheet
flowers_dict = fix_pickle()
#with open("flowers.pkl", 'rb') as fpkl:
#	pickle.load(fpkl)
#    flowers_dict = pickle.load(fpkl)

# Keys are types of flowers, printed here for reference
# No cosmos as of right now bc I completed them
#print(flowers_dict.keys())

# Given papa and mama flower, returns possible child flowers and their probabilities
def child_probas(flower1, flower2):
	flower1_genes = [fix_case(flower1.gene[2 * x] + flower1.gene[2 * x + 1]) for x in range(len(flower1.gene) // 2)]
	flower2_genes = [fix_case(flower2.gene[2 * x] + flower2.gene[2 * x + 1]) for x in range(len(flower2.gene) // 2)]
	ar1 = []
	ar2 = []
	child_dict = {}
	for x in range(len(flower1_genes)):
		carr = children(flower1_genes[x], flower2_genes[x])
		child_genes = [c[0] for c in carr]
		child_probas = [c[1] for c in carr]
		for x in range(len(child_genes)):
			if child_genes[x] in child_dict.keys():
				child_dict[child_genes[x]] = child_dict[child_genes[x]] + child_probas[x]
			else:
				child_dict[child_genes[x]] = child_probas[x]
		ar1.append(list(child_dict.keys()))
		ar2.append(list(child_dict.values()))
		child_dict = {}
	results = []
	if len(flower1_genes) < 4:
		for i in range(len(ar1[0])):
			for j in range(len(ar1[1])):
				for k in range(len(ar1[2])):
					results.append([fix_case(ar1[0][i])+fix_case(ar1[1][j])+fix_case(ar1[2][k]), ar2[0][i]*ar2[1][j]*ar2[2][k]])
	else:
		for i in range(len(ar1[0])):
			for j in range(len(ar1[1])):
				for k in range(len(ar1[2])):
					for l in range(len(ar1[3])):
						results.append([fix_case(ar1[0][i])+fix_case(ar1[1][j])+fix_case(ar1[2][k])+fix_case(ar1[3][l]), ar2[0][i]*ar2[1][j]*ar2[2][k]*ar2[3][l]])
	return results

# Changes rR to Rr
def fix_case(gene):
	if gene[1] < gene[0]:
		return gene[1] + gene[0]
	return gene

# Very literal children and probabilities genewise calculator
def children(gene1, gene2):
	return [[fix_case(gene1[0]+gene2[0]), 0.25],[fix_case(gene1[0]+gene2[1]), 0.25],[fix_case(gene1[1]+gene2[0]), 0.25],[fix_case(gene1[1]+gene2[1]), 0.25]]

# Given a flower and a string corresponsing to its species, will return a list of its possible parents
def get_parents(flower):
	flowers = flowers_dict[flower.species]
	ret = []
	for f in flowers.values():
		if f.isParent(flower):
			ret.append(f)
	return ret

# Enter a color and species and get a list of flower objects
def get_flowers_by_color(color, species):
	results = []
	for f in list(flowers_dict[species].values()):
		if (color in f.color):
			results.append(f)
	return results

# Enter a color and species and get a list of objects corresponding
# to flowers that could have created this flower
def get_parents_by_color(color, species):
	flowers = get_flowers_by_color(color, species)
	results = []
	for f in flowers:
		results.append(get_parents(f, species))
	return results

# Given a flower and species, will return a list of possible parents
# and the probability that those parents could have created this flower
def parent_probas(flower):
	results = []
	parents = get_parents(flower)
	for x in range(len(parents)):
		for y in range(x, len(parents)):
			probas_obj = [parents[x], parents[y], child_probas(parents[x], parents[y])]
			temp = [parents[x], parents[y], [x for x in probas_obj[2] if x[0] == flower.gene]]
			results.append(temp)
	df = pd.DataFrame.from_dict(results)
	df = preprocess(df)
	return results, df

def flower_by_id(flower_id, species):
	return flowers_dict[species][flower_id]

def heritage_data(flower):
  results = {}
  q = []
  q.append((flower,1))
  while len(q):
    temp = q.pop()
    temp2 = parent_probas(temp[0], flower.species)
    results[temp[0].gene] = temp2
    for x in temp2:
      if (x[0].gene not in results.keys()) and (not x[0].seed()):
        q.append((x[0],1))
      if ((x[1] != x[0]) and (x[1].gene not in results.keys()) and (not x[1].seed())):
        q.append((x[1],1))
  
  df = pd.DataFrame.from_dict(results[flower.gene])
  df = preprocess(df)
  return results, df

def preprocess(df):
  # Extract info from parents1 and parents2
  parent1 = [str(x) for x in df[0]]
  parent2 = [str(x) for x in df[1]]
  p1data = [str(df[0][x]).split("\t") for x in df[0].keys()]
  p2data = [str(df[1][x]).split("\t") for x in df[1].keys()]
  # Add extracted data to df
  parent1_id = [x[0] for x in p1data]
  parent1_gene = [x[1] for x in p1data]
  parent1_color = [x[2] for x in p1data]
  parent2_id = [x[0] for x in p2data]
  parent2_gene = [x[1] for x in p2data]
  parent2_color = [x[2] for x in p2data]
  df = df.assign(parent1_id = parent1_id)
  df = df.assign(parent1_gene = parent1_gene)
  df = df.assign(parent1_color = parent1_color)
  df = df.assign(parent2_id = parent2_id)
  df = df.assign(parent2_gene = parent2_gene)
  df = df.assign(parent2_color = parent2_color)

  # Extract probas info and add it to df
  probas = [x[0][1] if len(x) > 0 else 0 for x in df[2]]
  df = df.assign(probas = probas)
  df = df.drop([0, 1, 2], axis = 1)
  df = df.sort_values(by = 'probas', ascending = False)
  return df

def colorStuff(df):
	def highlight(s):
		return ['background-color: white'] * 2 + ['background-color: ' + s.parent1_color.split(' ')[0].lower()] + ['background-color: white'] * 2 + ['background-color: ' + s.parent2_color.split(' ')[0].lower()] + ['background-color: white']
	df = df.style.apply(highlight, axis=1)
	return df

def display(flower, df):
	print("PARENTS AND PROBABILITIES OF " + str(flower))
	df = colorStuff(df)
	return df

FileNotFoundError: ignored

In [ ]:
flower_dict = fix_pickle()

In [ ]:
green_mums = get_flowers_by_color("Green", "Mums")

In [ ]:
results, df = parent_probas(green_mums[0])
display(green_mums[0], df)

PARENTS AND PROBABILITIES OF 3c	RRYYWW	Green	Mums


,parent1_id,parent1_gene,parent1_color,parent2_id,parent2_gene,parent2_color,probas
33,3c,RRYYWW,Green,3c,RRYYWW,Green,1.000000
28,34,RRYyWW,Purple,3c,RRYYWW,Green,0.500000
19,1c,RrYYWW,Purple,3c,RRYYWW,Green,0.500000
34,3c,RRYYWW,Green,3d,RRYYWw,Green,0.500000
31,35,RRYyWw,Purple,3c,RRYYWW,Green,0.250000
6,14,RrYyWW,Yellow,3c,RRYYWW,Green,0.250000
29,34,RRYyWW,Purple,3d,RRYYWw,Green,0.250000
17,1c,RrYYWW,Purple,34,RRYyWW,Purple,0.250000
26,34,RRYyWW,Purple,34,RRYyWW,Purple,0.250000
24,1d,RrYYWw,Purple,3c,RRYYWW,Green,0.250000


In [ ]:
white_roses = get_flowers_by_color("White", "Roses")

In [ ]:
results, df = parent_probas(white_roses[9])
display(white_roses[9], df)

PARENTS AND PROBABILITIES OF 3c	rrYYwwss	White	Roses


,parent1_id,parent1_gene,parent1_color,parent2_id,parent2_gene,parent2_color,probas
81,3c,rrYYwwss,White,3c,rrYYwwss,White,1.000000
60,34,rrYYWwss,Yellow,3c,rrYYwwss,White,0.500000
89,3c,rrYYwwss,White,7c,RrYYwwss,Red,0.500000
35,1c,rrYywwss,Purple,3c,rrYYwwss,White,0.500000
82,3c,rrYYwwss,White,3d,rrYYwwSs,White,0.500000
68,34,rrYYWwss,Yellow,7c,RrYYwwss,Red,0.250000
87,3c,rrYYwwss,White,74,RrYYWwss,Orange,0.250000
90,3c,rrYYwwss,White,7d,RrYYwwSs,Pink,0.250000
31,1c,rrYywwss,Purple,1c,rrYywwss,Purple,0.250000
36,1c,rrYywwss,Purple,3d,rrYYwwSs,White,0.250000


In [ ]:
purple_tulips = get_flowers_by_color("Purple", "Pansies")
results, df = parent_probas(purple_tulips[0])
display(purple_tulips[0], df)

PARENTS AND PROBABILITIES OF 33	RRyyww	Purple	Pansies


,parent1_id,parent1_gene,parent1_color,parent2_id,parent2_gene,parent2_color,probas
30,33,RRyyww,Purple,33,RRyyww,Purple,1.000000
32,33,RRyyww,Purple,37,RRYyww,Purple,0.500000
27,31,RRyyWw,Red,33,RRyyww,Purple,0.500000
12,13,Rryyww,Blue,33,RRyyww,Purple,0.500000
35,37,RRYyww,Purple,37,RRYyww,Purple,0.250000
14,13,Rryyww,Blue,37,RRYyww,Purple,0.250000
23,17,RrYyww,Orange,33,RRyyww,Purple,0.250000
31,33,RRyyww,Purple,35,RRYyWw,Red,0.250000
5,11,RryyWw,Red,33,RRyyww,Purple,0.250000
29,31,RRyyWw,Red,37,RRYyww,Purple,0.250000


In [ ]:
keep = [x for x in df.values if ('Purple' not in x[2] and 'Purple' not in x[5])]
for x in keep:
  print(x)

['13' 'Rryyww' 'Blue' '13' 'Rryyww' 'Blue' 0.25]
['13' 'Rryyww' 'Blue' '31' 'RRyyWw' 'Red' 0.25]
['31' 'RRyyWw' 'Red' '31' 'RRyyWw' 'Red' 0.25]
['31' 'RRyyWw' 'Red' '35' 'RRYyWw' 'Red' 0.125]
['11' 'RryyWw' 'Red' '13' 'Rryyww' 'Blue' 0.125]
['17' 'RrYyww' 'Orange' '31' 'RRyyWw' 'Red' 0.125]
['13' 'Rryyww' 'Blue' '17' 'RrYyww' 'Orange' 0.125]
['11' 'RryyWw' 'Red' '31' 'RRyyWw' 'Red' 0.125]
['13' 'Rryyww' 'Blue' '35' 'RRYyWw' 'Red' 0.125]
['13' 'Rryyww' 'Blue' '15' 'RrYyWw' 'Orange' 0.0625]
['35' 'RRYyWw' 'Red' '35' 'RRYyWw' 'Red' 0.0625]
['11' 'RryyWw' 'Red' '17' 'RrYyww' 'Orange' 0.0625]
['11' 'RryyWw' 'Red' '35' 'RRYyWw' 'Red' 0.0625]
['11' 'RryyWw' 'Red' '11' 'RryyWw' 'Red' 0.0625]
['17' 'RrYyww' 'Orange' '35' 'RRYyWw' 'Red' 0.0625]
['15' 'RrYyWw' 'Orange' '31' 'RRyyWw' 'Red' 0.0625]
['17' 'RrYyww' 'Orange' '17' 'RrYyww' 'Orange' 0.0625]
['15' 'RrYyWw' 'Orange' '17' 'RrYyww' 'Orange' 0.03125]
['11' 'RryyWw' 'Red' '15' 'RrYyWw' 'Orange' 0.03125]
['15' 'RrYyWw' 'Orange' '35' 'RRYyWw' '